In [137]:
from cloudvolume import CloudVolume
import numpy as np
import torch
import imageio
from scipy.ndimage import binary_closing, binary_opening, binary_erosion, binary_fill_holes
from copy import deepcopy


In [3]:
cv = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/songyang_003/2401091513_sections1330_3748_0109_aligned/aligned_256_links", mip=[2048,2048,50], fill_missing=True, progress=False)

In [69]:
# z1398 = cv[:, :, 1398:1399, 0]
# imageio.imsave("z1398.png", z1398[:, :, 0].T.squeeze())
tissue_mask = imageio.imread("z1398_mask.png").T[0, :, :].reshape(873, 699, 1, 1)

/tmp/ipykernel_6746/3118965740.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  tissue_mask = imageio.imread("z1398_mask.png").T[0, :, :].reshape(873, 699, 1, 1)


In [17]:
imdata = cv[:,:,1330:3749,0] > 0 # (873, 699, 2419, 1)

In [38]:
imdata

VolumeCutout([[[[False],
                [False],
                [False],
                ...,
                [False],
                [False],
                [False]],

               [[False],
                [False],
                [False],
                ...,
                [False],
                [False],
                [False]],

               [[False],
                [False],
                [False],
                ...,
                [False],
                [False],
                [False]],

               ...,

               [[False],
                [False],
                [False],
                ...,
                [False],
                [False],
                [False]],

               [[False],
                [False],
                [False],
                ...,
                [False],
                [False],
                [False]],

               [[False],
                [False],
                [False],
                ...,
                [F

In [114]:
imdata_close = binary_closing(imdata, iterations=1, structure=np.ones((1,1,17,1)))

In [162]:
imdata_close_open = binary_opening(imdata_close, iterations=1, structure=np.ones((1,1,69,1)))

In [163]:
imdata_close_open = binary_opening(imdata_close_open, iterations=1, structure=np.ones((3,3,1,1)), border_value=1)
imdata_close_open = binary_closing(imdata_close_open, iterations=1, structure=np.ones((3,3,1,1)), border_value=1)

In [164]:
imdata_close_open = imdata_close_open * tissue_mask

In [168]:
erode_px = 64
imdata_close_open_erode = binary_erosion(imdata_close_open, iterations=erode_px, structure=np.ones((3,3,1,1)), border_value=1)
cv_mask = CloudVolume(f"gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close17_open69_erode_xy{erode_px}", mip=[2048,2048,50], delete_black_uploads=True, fill_missing=True, progress=False, info=deepcopy(cv.info))
cv_mask.info['scales'] = cv_mask.info['scales'][-1:]
cv_mask.info['scales'][-1]['encoding'] = 'raw'
cv_mask.info['scales'][-1]['chunk_sizes'] = [[873,699,1]]
cv_mask.commit_info()
cv_mask = CloudVolume(f"gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close17_open69_erode_xy{erode_px}", mip=[2048,2048,50], delete_black_uploads=True, fill_missing=True, progress=False, cdn_cache="no-cache")
cv_mask[:, :, 1330:3749, 0] = imdata_close_open_erode.astype(np.uint8) * 255

In [113]:
# cv_mask = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close19_open95", mip=[2048,2048,50], delete_black_uploads=True, fill_missing=True, progress=False, info=deepcopy(cv.info))
# cv_mask.info['scales'] = cv_mask.info['scales'][-1:]
# cv_mask.info['scales'][-1]['encoding'] = 'raw'
# cv_mask.info['scales'][-1]['chunk_sizes'] = [[873,699,1]]
# cv_mask.commit_info()
# cv_mask = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close19_open95", mip=[2048,2048,50], delete_black_uploads=True, fill_missing=True, progress=False, cdn_cache="no-cache")
# cv_mask[:, :, 1330:3749, 0] = imdata_close_open

In [128]:
cv = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/songyang_003/2401091513_sections1330_3748_0109_aligned/aligned_256_links", mip=[2048,2048,50], fill_missing=True, progress=False)
cv_mask_red = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close17_open69_erode_xy64", mip=[2048,2048,50], fill_missing=True, progress=False)
cv_mask_orange = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close17_open69_erode_xy32", mip=[2048,2048,50], fill_missing=True, progress=False)
cv_mask_yellow = CloudVolume("gs://zetta_jchen_mouse_cortex_001_drop/nkem/2401091513_sections1330_3748_0109_aligned/tissue_mask_close17_open69_erode_xy16", mip=[2048,2048,50], fill_missing=True, progress=False)

data = cv[:,:,1330:3749,0] > 0
mask_red = cv_mask_red[:,:,1330:3749,0] > 0
mask_orange = cv_mask_orange[:,:,1330:3749,0] > 0
mask_yellow = cv_mask_yellow[:,:,1330:3749,0] > 0


In [138]:
data = binary_fill_holes(data, structure=np.ones((3,3,1,1)))

# cc3d.connected_components(data.astype(np.uint8).squeeze(), connectivity=4)

In [139]:
data

array([[[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],

        ...,

        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]]],


       [[[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [False]],

        [[False],
         [False],
         [False],
         ...,
         [False],
         [False],
         [Fa

In [140]:
intersection_counts = {}
for mask_name, mask in [("red", mask_red), ("orange", mask_orange), ("yellow", mask_yellow)]:
    intersection = np.logical_and(~data, mask)
    intersection_count = np.sum(intersection, axis=(0,1))
    intersection_counts[mask_name] = intersection_count

In [150]:
import itertools
ranges = []
START_Z = 1330
for key, group in itertools.groupby(enumerate(intersection_counts['orange']), lambda i_x:i_x[1] > 0):
    if key:
        group = list(group)
        ranges.append(range(group[0][0], group[-1][0]))

print(sorted(ranges, key=len, reverse=True))

[range(988, 1007), range(541, 556), range(892, 905), range(908, 919), range(753, 762), range(863, 872), range(461, 469), range(1461, 1469), range(532, 539), range(734, 741), range(743, 750), range(957, 964), range(1451, 1458), range(207, 213), range(579, 585), range(618, 624), range(980, 986), range(1242, 1248), range(35, 40), range(120, 125), range(454, 459), range(677, 682), range(784, 789), range(944, 949), range(971, 976), range(181, 185), range(200, 204), range(368, 372), range(559, 563), range(569, 573), range(591, 595), range(1107, 1111), range(1250, 1254), range(1345, 1349), range(2395, 2399), range(107, 110), range(398, 401), range(423, 426), range(474, 477), range(657, 660), range(804, 807), range(849, 852), range(858, 861), range(921, 924), range(932, 935), range(1117, 1120), range(1141, 1144), range(1178, 1181), range(1234, 1237), range(1265, 1268), range(1501, 1504), range(1714, 1717), range(1957, 1960), range(2321, 2324), range(10, 12), range(27, 29), range(67, 69), range

In [151]:
data = intersection_counts['orange']
triplets = [(range(i, i+1), int(data[i:i+1].sum())) for i in range(0, len(data), 1)]
sorted_triplets = sorted(triplets, key=lambda x: x[1], reverse=True)

filtered_triplets = [(range(triplet.start, triplet.stop), count) for (triplet, count) in sorted_triplets if count > 0]
print(len(filtered_triplets))
print(filtered_triplets)

919
[(range(910, 911), 231473), (range(933, 934), 230849), (range(934, 935), 230849), (range(893, 894), 230676), (range(895, 896), 230676), (range(897, 898), 230676), (range(899, 900), 230676), (range(904, 905), 230676), (range(872, 873), 230580), (range(128, 129), 229910), (range(858, 859), 229431), (range(861, 862), 229431), (range(867, 868), 227823), (range(817, 818), 226608), (range(997, 998), 225105), (range(939, 940), 224829), (range(252, 253), 224528), (range(807, 808), 224015), (range(42, 43), 223455), (range(45, 46), 223455), (range(97, 98), 223364), (range(27, 28), 223099), (range(32, 33), 223099), (range(40, 41), 223099), (range(960, 961), 221610), (range(944, 945), 221556), (range(945, 946), 221556), (range(949, 950), 221556), (range(967, 968), 221419), (range(954, 955), 221249), (range(321, 322), 221165), (range(957, 958), 219693), (range(958, 959), 219693), (range(442, 443), 219146), (range(948, 949), 212614), (range(892, 893), 202805), (range(10, 11), 185125), (range(952

In [143]:
triplets[1446]

(range(1446, 1447), 0)

In [152]:
filtered_triplets.sort(key=lambda x: x[0].start)
candidate = filtered_triplets[0]
merged = []
for current in filtered_triplets[1:]:
    if current[0].start <= candidate[0].stop:
        candidate = (range(candidate[0].start, current[0].stop), int(intersection_counts["orange"][candidate[0].start:current[0].stop].sum()))
    else:
        merged.append((range(candidate[0].start + 1330, candidate[0].stop + 1330), candidate[1]))
        candidate = current

merged_sorted = sorted(merged, key=lambda x: x[1], reverse=True)
merged_sorted_filtered = [x for x in merged_sorted if x[0].stop - x[0].start > 1]

print(len(merged_sorted_filtered))
print(merged_sorted_filtered)



188
[(range(2222, 2236), 1440983), (range(2318, 2338), 1049152), (range(2274, 2280), 1022742), (range(2287, 2295), 688567), (range(2262, 2266), 660183), (range(2572, 2579), 611866), (range(2193, 2203), 464529), (range(2188, 2192), 459219), (range(2437, 2442), 457526), (range(2238, 2250), 387045), (range(1357, 1360), 373569), (range(2471, 2475), 359947), (range(1340, 1343), 338283), (range(2258, 2261), 331381), (range(3651, 3655), 307333), (range(2373, 2375), 301554), (range(1365, 1371), 300128), (range(2564, 2568), 288038), (range(2508, 2512), 287244), (range(2997, 3000), 284393), (range(1871, 1887), 282654), (range(2764, 2767), 277617), (range(2552, 2555), 241155), (range(1558, 1560), 239455), (range(2634, 2637), 237075), (range(1457, 1459), 236727), (range(1427, 1430), 230743), (range(2267, 2270), 230021), (range(2807, 2809), 227940), (range(2146, 2148), 226700), (range(1581, 1583), 225029), (range(2296, 2299), 224623), (range(2134, 2138), 224162), (range(2390, 2393), 223191), (range

In [153]:
with open("orange.txt", "w") as f:
  for i in merged_sorted_filtered:
    f.write(f"{i[0].start}\t{i[0].stop-1}\t{i[1]}\n")